In [2]:
# Import of important libraries
### /home1/08289/tg875885/radial_to_rotating_flows/Aharon/Trident ###
import numpy as np
import yt
import trident
import unyt
import pylab as pl
from astropy import units as un, constants as cons
from numpy import log10 as log
import h5py
from scipy.spatial.transform import Rotation

# Parameters

In [1]:
# Filetree, snapshot directories and its redshift
SnapshotNumber=206
Snapshot_fps=('/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.0.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.1.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.2.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.3.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.4.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.5.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.6.hdf5',
              '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12b_r7100/output/snapdir_206/snapshot_206.7.hdf5') # Snapshot Files
print("The snapshot number is:",SnapshotNumber)
ray_fp='trident_ray.h5' # The name of the ray that will be created
z=1.5819671420461177 # The redshift of the given snapshot
print("The redshift of the given snapshot is: z=",z)
a=(z+1)**-1 # The scale factor for this redshift
print("The scale factor of the given snapshot is: a=",a)

The snapshot number is: 206
The redshift of the given snapshot is: z= 1.5819671420461177
The scale factor of the given snapshot is: a= 0.3873015979620621


In [3]:
# Manual input of halo data parameters (units are kpc)
Rvir=71.06209719407916 # The virial radius of the given snapshot
print("The virial radius is:",Rvir,"kpc")
HaloCenterCoordinates=np.array([28688.8551165,29566.0848924,28735.075096]) # The center coordinates of the halo
print("The center coordinates are:",HaloCenterCoordinates)
Jvector=np.array([0.6493,-0.6628,-0.373]) # The angular momentum vector of the halo
print("The angular momentum is:",Jvector)

The virial radius is: 71.06209719407916 kpc
The center coordinates are: [28688.8551165 29566.0848924 28735.075096 ]
The angular momentum is: [ 0.6493 -0.6628 -0.373 ]


In [4]:
# Insert the snapshot files into a list and calculate the hubble parameter for the given halo
fs=[h5py.File(Snapshot_fp) for Snapshot_fp in Snapshot_fps] # List of HDF5 files of the snapshot
HubbleParameter=fs[0]['Header'].attrs['HubbleParam'] # Hubble parameter of the given snapshot
print("The hubble parameter of the given snapshot is:",HubbleParameter)

The hubble parameter of the given snapshot is: 0.702


In [5]:
# The angular momentum vector
Jvector=Jvector/np.linalg.norm(Jvector) # Normalize the angular momentum vector
print("The normalized J vector is:",Jvector)
rotvec=np.cross(Jvector,np.array([0,1,0])) # Cross between the angular momentum vector and the vector (0,1,0)
print("The cross product of J vector with (0,1,0) is:",rotvec)
sin_theta=np.linalg.norm(rotvec) # The norm of the crossed vector
rotvec_normed=-rotvec/sin_theta*np.arcsin(sin_theta) # Normalize the crossed vector and multiply it by arcsin of the norm 
                                                     # The minus sign is to be consistent with direction bug
print("The normalized crossed product multiply by arcsin of the norm is:",rotvec_normed)
rot1=Rotation.from_rotvec(rotvec_normed) # An object containing the rotations is represented by rotvec_normed vector 

The normalized J vector is: [ 0.64929243 -0.66279227 -0.37299565]
The cross product of J vector with (0,1,0) is: [ 0.37299565 -0.          0.64929243]
The normalized crossed product multiply by arcsin of the norm is: [-0.42153837  0.         -0.73379319]


In [6]:
# x hat and y hat vectors of rot1 vector and compariosn to J vector
x_hat=rot1.apply(np.array([1,0,0]),inverse=True) # Direction vector in x axis
y_hat=rot1.apply(np.array([0,1,0]),inverse=True) # Direction vector in y axis
print("x_hat vector is:",x_hat)
print("y_hat vector is:",y_hat)
print("J vector is:",Jvector)
if(np.dot(x_hat,y_hat)==0):
    print("x_hat and y_hat are orthogonal")
else:
    print("x_hat and y_hat are not orthogonal")
print("y_hat should be the same as J vector, but they are not due to direction bug in projection plot in ICV paper")

x_hat vector is: [0.74646222 0.64929243 0.14564853]
y_hat vector is: [-0.64929243  0.66279227  0.37299565]
J vector is: [ 0.64929243 -0.66279227 -0.37299565]
x_hat and y_hat are not orthogonal
y_hat should be the same as J vector, but they are not due to direction bug in projection plot in ICV paper


In [7]:
# Sightline parameters
# Sightline is impact_parameter offset from the center along the z-axis, runs perpendicular to the z-axis, and has a length of path_length
path_length=0.4*Rvir
print("The path length is:",path_length,"kpc")
d=path_length/2
print("The half path length is:",d,"kpc")

The path length is: 28.424838877631665 kpc
The half path length is: 14.212419438815832 kpc


In [8]:
# Extract the particles coordinates data from all snapshots parts
coords=np.concatenate([f['PartType0']['Coordinates'][:] for f in fs],axis=0)/HubbleParameter*a-HaloCenterCoordinates # Merge all the particles coordinates relative to the center of the halo to one array
print("The particles coordinates are:\n",coords)
print("")
# Rotate the particles coordinates
rot_coords=rot1.apply(coords)
print("The particles rotated coordinates are:\n",rot_coords)

The particles coordinates are:
 [[-12860.85350781 -13253.90430622 -12882.83915349]
 [-12860.89342294 -13253.88176987 -12882.84385345]
 [-12860.90163144 -13253.90971789 -12882.84408847]
 ...
 [-13294.99680124 -12662.54006754 -12991.99628828]
 [-13294.45797538 -12665.82852627 -12989.04426141]
 [-13300.3063791  -12664.13109793 -12991.32257816]]

The particles rotated coordinates are:
 [[-20082.16757427  -5239.37347886  -8734.44835295]
 [-20082.18342127  -5239.33437841  -8734.46687921]
 [-20082.20772927  -5239.34766009  -8734.45786562]
 ...
 [-20038.16939744  -4606.25101765  -9118.28093577]
 [-20039.47239727  -4607.67934503  -9114.27084487]
 [-20043.06771583  -4603.60678065  -9117.84347974]]


In [9]:
# Extract the particles internal energy data from all snapshots parts and calculate their Ts
epsilons=np.concatenate([f['PartType0']['InternalEnergy'][:] for f in fs],axis=0)[:]
print("The particles internal energies are:\n",epsilons)
print("")
Ts=(un.km**2/un.s**2*cons.m_p/cons.k_B).to('K').value*(2./3*0.62)*epsilons 
print("The particles Ts are:\n",Ts)

The particles internal energies are:
 [ 76.73263     4.1838555 209.86205   ... 125.18579   120.488144
 123.674385 ]

The particles Ts are:
 [ 3842.333     209.50366 10508.696   ...  6268.592    6033.3604
  6192.909  ]


In [10]:
# Extract more data of the particles
ls=np.concatenate([f['PartType0']['SmoothingLength'][:] for f in fs],axis=0)[:]
print("The particles smoothing lengths are:\n",ls)
print("")
Ms=np.concatenate([f['PartType0']['Masses'][:] for f in fs],axis=0)[:]
print("The particles masses are:\n",Ms)
print("")
rhos=np.concatenate([f['PartType0']['Density'][:] for f in fs],axis=0)[:]
print("The particles densities are:\n",rhos)

The particles smoothing lengths are:
 [ 0.08457942  0.04467753  0.04911519 ... 18.682768   18.685026
 18.815834  ]

The particles masses are:
 [5.8387161e-07 5.7199185e-07 5.8684327e-07 ... 4.9612282e-07 4.9612282e-07
 4.9612282e-07]

The particles densities are:
 [7.3290970e-03 4.9030788e-02 3.7853215e-02 ... 5.8099098e-10 5.8075023e-10
 5.6855037e-10]


In [21]:
# Calculate the impact parameter, z_hat, z_offset, and the sightline start, center, and end point
impact_parameter=100
print("The impact parameter is:",impact_parameter,"kpc")
z_hat=np.cross(x_hat,y_hat)
print("z_hat is:",z_hat)
z_offset=-2*z_hat
print("z_hat offset is:",z_offset)
sl_center=HaloCenterCoordinates+impact_parameter*y_hat+z_offset
print("The sightline center point is:",sl_center)
sl_start=sl_center-path_length/2.*x_hat
print("The sightline start point is:",sl_start)
sl_end=sl_center+path_length/2.*x_hat
print("The sightline end point is:",sl_end)

The impact parameter is: 100 kpc
z_hat is: [ 0.14564853 -0.37299565  0.91633005]
z_hat offset is: [-0.29129706  0.7459913  -1.83266009]
The sightline center point is: [28623.63457683 29633.11011056 28770.54200081]
The sightline start point is: [28613.02554262 29623.88209426 28768.47198281]
The sightline end point is: [28634.24361105 29642.33812685 28772.61201881]


# Generate Mock Observation

In [22]:
# Load the simulation data
ds=yt.load(Snapshot_fps[0])

yt : [INFO     ] 2023-04-29 05:15:45,416 Calculating time from 3.873e-01 to be 1.321e+17 seconds
yt : [INFO     ] 2023-04-29 05:15:45,417 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2023-04-29 05:15:45,464 Parameters: current_time              = 1.3214444680781424e+17 s
yt : [INFO     ] 2023-04-29 05:15:45,465 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-04-29 05:15:45,465 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-04-29 05:15:45,466 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2023-04-29 05:15:45,466 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-04-29 05:15:45,466 Parameters: current_redshift          = 1.5819671420461177
yt : [INFO     ] 2023-04-29 05:15:45,467 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2023-04-29 05:15:45,467 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2023-04-29 05:15:45,467 Parameters: omega_radiation

In [23]:
# Get the simulation kpc unit, this avoids a bug that sometimes pops up
kpc=ds.quan(1,'kpc')

In [24]:
# Other parameters
line_list=['H'] # ['H', 'C', 'N', 'O','Mg','Si']

In [25]:
ray = trident.make_simple_ray(ds,
                              start_position=sl_start*kpc,
                              end_position=sl_end*kpc,
                              data_filename=ray_fp,
                              lines=line_list)

yt : [INFO     ] 2023-04-29 05:15:47,559 Allocating for 1.559e+08 particles
Initializing coarse index : 100%|███████| 288/288 [00:20<00:00, 13.75it/s]
yt : [INFO     ] 2023-04-29 05:16:08,514 Updating index_order2 from 2 to 2
Initializing refined index: 100%|███████| 288/288 [00:58<00:00,  4.93it/s]
yt : [INFO     ] 2023-04-29 05:17:08,948 Getting segment at z = 1.5819671420461177: [0.86437133 0.89490832 0.86906722] unitary to [0.86501231 0.89546585 0.86919228] unitary.
yt : [INFO     ] 2023-04-29 05:17:08,949 Getting subsegment: [0.86437133 0.89490832 0.86906722] unitary to [0.86501231 0.89546585 0.86919228] unitary.


RuntimeError: No zones along light ray with nonzero ('gas', 'temperature'). Please modify your light ray trajectory.

In [ ]:
a = ds.scale_factor
dlambda = 0.01
spectral_resolutions = (2000,20000,200000)
vs,flux = [None]*len(spectral_resolutions), [None]*len(spectral_resolutions)
for i,spectral_resolution in enumerate(spectral_resolutions):
    wl = 1206 #wavelength[1]
    sg = trident.SpectrumGenerator(lambda_min = 1100/a,lambda_max = 1300/a,dlambda=dlambda, bin_space='wavelength')
    sg.make_spectrum(ray,lines=['Si III'],store_observables = True)
    lsf_width = wl/spectral_resolution / dlambda
    sg.apply_lsf(function='gaussian',width=lsf_width)
    sg.save_spectrum( 'yoyo4.h5' )
    f = h5py.File('yoyo4.h5')
    vs[i], flux[i] = (f['wavelength'][:]*a/wl - 1)*3e5,f['flux'][:]
    f.close()

In [ ]:
sg.line_observables_dict['Si III 1206']

# Plot Ray Properties

In [ ]:
# Get property fields
temp = ray.r[('gas', 'temperature')]
dens = ray.r[('gas','H_nuclei_density')]
#metallicity = ray.r[('gas', 'O_metallicity')]
fsiIII = ray.r[('gas', 'Si_p2_ion_fraction')]
vCenter = np.array([-58.64,82.05,86.61])
vCenter_los = np.dot(vCenter,x_hat)*unyt.km/unyt.s
vlos = ray.r[('gas','velocity_los')].to('km/s') + vCenter_los

In [ ]:
# Get coordinates and convert to relevant frame
coords = np.array([ ray.r[('gas', _)].to( 'kpc' ) for _ in ( 'x', 'y', 'z' ) ]).transpose()
coords -= sl_center
d_along_sightline = np.dot( coords, x_hat )

In [ ]:
pl.plot(d_along_sightline,ray.r[('gas','velocity_los')].to('km/s'))
pl.axhline(-vCenter_los)

In [ ]:
import matplotlib
from matplotlib import gridspec

In [ ]:
fig = pl.figure(figsize=(7,4))
gs = gridspec.GridSpec(nrows=2,ncols=5,wspace=0.7)
ax = fig.add_subplot(gs[0,2:])
ax.plot(d_along_sightline,temp,lw=0.3,c='k')
ax.set_yscale( 'log' )
ax.set_xlim(-30,30)
pl.ylim(0.8e4,7e6)
ax = fig.add_subplot(gs[1,2:])
ax.plot(d_along_sightline,fsiIII,lw=0.3,c='k')
ax.set_xlim(-30,30)
ax.set_yscale( 'log' )
pl.ylim(1e-3,1)
ax.set_xlabel( 'distance along line-of-sight [kpc]',fontsize=9 )
ax = fig.add_subplot(gs[1,:2])
for i,spectral_resolution in enumerate(spectral_resolutions):
    pl.plot(vs[i],flux[i],label=r'R=%d'%spectral_resolution)
pl.xlim(-200,450)
pl.legend(fontsize=8,handlelength=1,frameon=False)
ax.set_xlabel( 'velocity [km/s]',fontsize=9 )
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.5))
pl.savefig( 'sightline2.pdf' )